In [ ]:
import cv2
import numpy as np

def find_3d_point(img_left_path, img_right_path, baseline):
    # Load images
    img_left = cv2.imread(img_left_path)
    img_right = cv2.imread(img_right_path)

    # Stereo rectification parameters (calibrated beforehand)
    # These parameters should be obtained during the stereo calibration process
    # You need to replace these with your actual parameters
    camera_matrix_left = np.array([[fx_left, 0, cx_left],
                                   [0, fy_left, cy_left],
                                   [0, 0, 1]])

    dist_coeffs_left = np.array([k1_left, k2_left, p1_left, p2_left, k3_left])

    camera_matrix_right = np.array([[fx_right, 0, cx_right],
                                    [0, fy_right, cy_right],
                                    [0, 0, 1]])

    dist_coeffs_right = np.array([k1_right, k2_right, p1_right, p2_right, k3_right])

    R, T, _, _, _, _, _ = cv2.stereoRectify(camera_matrix_left, dist_coeffs_left,
                                            camera_matrix_right, dist_coeffs_right,
                                            img_left.shape[::-1], R, T, flags=cv2.CALIB_ZERO_DISPARITY)

    # Undistort and rectify the images
    img_left_rect = cv2.remap(img_left, cv2.initUndistortRectifyMap(camera_matrix_left, dist_coeffs_left, R, camera_matrix_left, img_left.shape[::-1], cv2.CV_16SC2), cv2.INTER_LINEAR)
    img_right_rect = cv2.remap(img_right, cv2.initUndistortRectifyMap(camera_matrix_right, dist_coeffs_right, R, camera_matrix_right, img_right.shape[::-1], cv2.CV_16SC2), cv2.INTER_LINEAR)

    # Stereo matching and disparity map calculation
    stereo = cv2.StereoBM_create(numDisparities=16, blockSize=15)
    disparity = stereo.compute(cv2.cvtColor(img_left_rect, cv2.COLOR_BGR2GRAY), cv2.cvtColor(img_right_rect, cv2.COLOR_BGR2GRAY))

    # Triangulate to get 3D point
    points_3d = triangulate_points(disparity, camera_matrix_left, T, baseline)

    return points_3d

def triangulate_points(disparity, camera_matrix, translation, baseline):
    # Triangulate points from disparity map
    points_3d = cv2.reprojectImageTo3D(disparity, Q(camera_matrix, translation, baseline))
    return points_3d

def Q(camera_matrix, translation, baseline):
    # Calculate the Q matrix for triangulation
    return np.array([[1, 0, 0, -camera_matrix[0, 2]],
                     [0, 1, 0, -camera_matrix[1, 2]],
                     [0, 0, 0, camera_matrix[0, 0]],
                     [0, 0, -1/baseline, translation[0, 0]]])

# Path to left and right images
left_image_path = 'path/to/left_image.jpg'
right_image_path = 'path/to/right_image.jpg'

# Baseline distance between the two cameras
baseline = 0.1  # Replace with the actual baseline distance in meters

# Find 3D point
point_3d = find_3d_point(left_image_path, right_image_path, baseline)

# Print or use the 3D coordinates of the point
print("3D Point:", point_3d)
